# Calculate CMIP Indeces

This file 
1. calculates all the indeces for all months for all cesmFullForcingFiles.



In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils.timePeriod as tp



In [2]:
import xarray
import matplotlib.pyplot as plt

# 1. Calculate CESM Indeces

import warnings
warnings.filterwarnings('ignore')

For each experiment in historical, calculate indices

In [3]:
timePIndeces=dict()
experiment='past1000'


for model in _model.past1000:
    print(experiment)
    
    fxDs = fh.loadModelData(model, 'areacella_fx', '') #is the resolution the same in all th experiments?
    #SST
    tsDs = fh.loadModelData(model, 'ts', 'piControl')
    controlDs=xarray.merge([tsDs.ts, fxDs.areacella])
    controlDs=controlDs.assign_attrs({'project_id':'CMIP5'})
    
    sstClimat=sst.calculateClimatology(controlDs)
    
    tsDs = fh.loadModelData(model, 'ts', experiment)
    sstDs = xarray.merge([tsDs.ts, fxDs.areacella])
    sstDs=sstDs.assign_attrs({'project_id':'CMIP5'})

    sstIndex = sst.calculateIndex(sstDs, sstClimat) #(reducing the model set at this step could save time?)
    
    #psl
    
    pslDs = fh.loadModelData(model, 'psl', experiment)
    pslIndex, junk = psl.calculateSamIndex(pslDs, pslClimat)
    indeces = xarray.merge([pslIndex, sstIndex])
    indeces.assign_attrs(climatology='calculated based on 850-2005 climatology from the control run')
    indeces
    #indeces.to_netcdf('results/cmipMonthlyIndeces/' + experiment + '.nc')

    

    


past1000
/Volumes/Untitled/CMIP5-PMIP3/CMIP5/areacella_fx/


ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation

In [ ]:
#loading both those things
for keys in sstClimat:
    sstClimat[keys].close()
    
pslClimat.close()

This grabs the monthly CESM indeces, and calculates the averages for the time periods of interest (aka the warm season), as set in the _indexDefitionions file

Theres a few examples of output at the bottom to sanity check this is meaningful

In [ ]:
#experimentSet = ['LME.002']
experimentSet = _model.cesmAll

The indeces for each month have already been calculated. So loading up the saved versions, and putting them into a single xarray.

In [ ]:
timePIndeces=list()

for experiment in experimentSet:
    print(experiment)
    indecesDs = xarray.open_dataset('results/cesmMonthlyIndeces/' + experiment + '.nc')

    timePIndeces.append(tp.averageForTimePeriod(indecesDs))


results=xarray.concat(timePIndeces, 'experiment')


results['experiment']=experimentSet

results=results.assign_attrs({**indecesDs.attrs,**_index.monthsOfInterest})

Save the result for use later

In [ ]:
results.to_netcdf('results/cesmTradIndeces.nc')

Some sanity checks to see if it looks ok:


In [ ]:
results

In [ ]:
# plotSomething
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)

#results.sel(experiment='001', index='nino34').plot(label='001')
results.nino34.sel(experiment='LME.002').plot(label='LME.001')
results.nino34NoDetrend.sel(experiment='LME.002').plot(label='LME.001')

#Tidy up a bit
plt.title("CESM Full Forcing Nino34 Ensemble")
plt.ylim(-4,4)
plt.minorticks_on()
plt.legend()

In [ ]:
# A multi-experiment mean?

mean = results.sam.mean(dim='experiment')
std = results.sam.std(dim='experiment')

In [ ]:
plt.figure()
mean.plot()
std.plot()

In [ ]:
# why limit yourself to one index?

mean = results.mean(dim='experiment')
std = results.std(dim='experiment')

In [ ]:
std

# 2. Calculate Compound Years for CESM

In [ ]:
# Here is the data
indecesDs = xarray.open_dataset('results/cesmTradIndeces.nc')

indecesDs.experiment.values

In [ ]:
indecesDs

# Define a criteria to determine events

So we are going to define events as follows:
- For SST indeces (aka Enso and Iod), this is 1 sigma above/below mean
- For PSL indeces (aka Sam) this is 0.5 sigma above/below mean

For now, using a 850-1850 climatology

In [ ]:
# This should calculate stds for all experiments and all indeces
stdDa = indecesDs.std(dim='year')

# define some criteria that we want to use to identifiy it thats an event
criteriaDa= xarray.merge(
    (stdDa[['nino34','dmi']], stdDa['sam']/2)
)

# Calculate the compounding years and plot an example

In [ ]:
%run utils/compound.ipynb



firePos, fireNeg = applyCriteria(indecesDs, criteriaDa)

firePos=compound(firePos)

fireNeg=compound(fireNeg)

firePos.to_netcdf('results/cesmFirePos.nc')

fireNeg.to_netcdf('results/cesmFireNeg.nc')

In [ ]:
firePos

In [ ]:
expToPlot = 'LME.002'

toPlot=3*firePos.all3.sel(experiment=expToPlot)

plt.figure(figsize=(12,12))

plt.vlines(
    x=firePos.year, 
    ymin=0,
    ymax=toPlot,
    label='all3'
)

colors=['orange', 'purple', 'green']
iCol = 0

for iPair in firePos.attrs['pairs']:
    plt.vlines(
        x=firePos.year, 
        ymin=0,
        ymax=2*firePos[iPair].sel(experiment=expToPlot),
        color=colors[iCol],
        label=iPair
    )
    iCol+=1
plt.legend()
plt.xlim(1700,2100)
plt.title(expToPlot+': Pairs and all-three compounds for individal years')